In [10]:
from scipy.constants import h, c, hbar, u, k 
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import itertools
import math as m
from ipynb.fs.full.Conversions_Epimetheus import *
from tqdm.contrib.itertools import product
from tqdm import tqdm

# Functions [0]

# Epimetheus 

## [E.1] Key and Distance/Angle between atoms

This function returns an array composed of the distances between molecules, the angle between molecules and the key parameter of how many atoms the molecule is composed of.

You need to currently input the coordinates of the atoms in a molcule in Angstrom [Å] units.

The output (distance between atoms) is returned in Angstrom [Å] units. (Also return the original coordinates for verification purposes).

In [1]:
def Key_and_Pos(matrix_of_coordinates):
    
    # **** Input ****
    # Positions: angstroms [Å]
    
    # Shouldn't matter on format of inputted (list/array)
    if isinstance(matrix_of_coordinates, list) == True:
        coordinates = np.copy(matrix_of_coordinates)
        coordinates = np.array(coordinates)
    elif isinstance(matrix_of_coordinates, list) == False:
        coordinates = matrix_of_coordinates
        
    # **** For hypothetical conversions down the line ****    
    #coordinates = coordinates * angstrom_to_bohr
    
    # **** DIATOMIC ****
    if len(coordinates) == 2:
        #print("Diatomic")
        #Number of atoms
        N = 2
        dia_co = np.copy(coordinates)
        r_m1_m2 = abs(dia_co[0] - dia_co[1])
        #print(r_m1_m2)
        r_m1_m2_mag = np.linalg.norm(r_m1_m2)
        #print(r_m1_m2_mag)
        
        Key = np.array([int(N), r_m1_m2_mag])
        #print("Key; N =", Key[0], ", r_m1_m2 magnitude=", Key[1])
    
    # **** TRIATOMIC ****
    # Treating central atom as the "origin"
    elif len(coordinates) == 3:
        #print("Triatomic")
        #Number of molecules
        N = 3
        tria_co = np.copy(coordinates)
        #print("tria_co",tria_co)
        #print("[0]", tria_co[0])
        
        #Geometry between m1 and m2
        r_m1_m2 = (tria_co[0] - tria_co[1])
        #print("Distance between m1 & m2", r_m1_m2)
        r_m1_m2_mag = np.linalg.norm(r_m1_m2)
        #print("Magnitude between m1 & m2", r_m1_m2_mag)
        
        #Geometry between m2 and m3
        r_m2_m3 = (tria_co[2] - tria_co[1])
        #print("Distance between m2 & m3", r_m2_m3)
        r_m2_m3_mag = np.linalg.norm(r_m2_m3)
        #print("Magnitude between m2 & m3", r_m2_m3_mag)
        
        #Geometry between m1 and m3
        r_m1_m3 = (tria_co[2] - tria_co[0])
        #print("Distance between m1 & m3", r_m1_m3)
        r_m1_m3_mag = np.sqrt(np.dot(r_m1_m3, r_m1_m3))
        #print("Magnitude between m1 & m3", r_m1_m3_mag)
        
        #Dot and product
        dot = np.dot(r_m1_m2, r_m2_m3)
        product = r_m1_m2_mag * r_m2_m3_mag
        
        #Angle
        costheta = dot/product
        
        if costheta < -1:
            costheta = -1 + (-1* (costheta+1))
        elif costheta > 1:
            costheta = 1 - ((costheta-1))
        else:
            costheta = costheta
        
        theta = np.arccos(costheta)
        #print(np.rad2deg(theta))
        Key = np.array([int(N), r_m1_m2_mag, r_m2_m3_mag, r_m1_m3_mag, theta])
        """print("Key; N =", Key[0], 
              ", r_m1_m2 magnitude=", Key[1], 
              ", r_m2_m3 magnitude=", Key[2], 
              ", r_m1_m2 magnitude=", Key[3],
              ", theta =", Key[4])"""
        
    
    # **** POLYATOMIC ****
    elif len(coordinates) > 3:
        print("Polyatomic")
        poly_co = np.copy(coordinates)
        print("Polyatomic not currently coded")
    
    # **** PRINT STATEMENTS ****
    #print("Key =")
    #print(Key)
    #print("----------------------------------")
    #print("Positions [Å] =")
    #print(coordinates)
    #print("----------------------------------")
        
    # **** OUTPUT ****     
    # Angstrom [Å]
    
    return Key, coordinates

.

## [E.2] Creating the Hessian matrix 

$\eta_{ii}$ = $\frac{1}{12h^2}\left(-E_{\left(-2,0\right)} + 16E_{\left(-1,0\right)} - 30E_{\left(0,0\right)} + 16E_{\left(1,0\right)} - E_{\left(2,0\right)}\right)$

$\eta_{ij}$ = $\frac{1}{4h^2}\left(E_{\left(-1,-1\right)} - E_{\left(-1,1\right)} - E_{\left(1,-1\right)} + E_{\left(1,1\right)}\right)$

The function to construct the Hessian matrix from which the mode frequencies can ultimately be derived.

Requires the input of "Key" from the previous function and the coordinates of the molecule.

PES Function outputs specific Energy unit, e.g. Hartrees - This unit will be used throughout the whole document

Output: $\eta_{ii}$ = $\frac{\left[Hartrees \right]}{ \left[Bohrs\right]^2}$

In [2]:
# Function to calculate the Cartesian Hessian
def Hess_CART(Key, Pos, PES, molecule):
    
    # Initialisation of Hessian and Dimensions
    # This work irregardless of molecule size
    dimension = int(Key[0])
    # print(dimension)
    Hessian = np.zeros(((3*dimension), (3*dimension)))
    
    hessunit = 0.005 
    # This can be changed if better value found
    # Hessunit is in bohrs [Bohr]
    
    unit_sort = bohr_to_angstrom # change as appropriate - current PES requires Å
    
    i = np.arange(0, (3*dimension))
    j = np.arange(0, (3*dimension))
    
    Pos = np.copy(Pos)
    Pos = Pos.reshape(1, dimension*3)
    
    for i,j in product(i, j):
            
        if i != j:
            #E_minusH_minusH
            E_minusH_minusH = np.copy(Pos)
            E_minusH_minusH[0][i] = E_minusH_minusH[0][i] - (hessunit * unit_sort)
            E_minusH_minusH[0][j] = E_minusH_minusH[0][j] - (hessunit * unit_sort)
            E_minusH_minusH = E_minusH_minusH.reshape(dimension,3)
            E_minusH_minusH_key, E_minusH_minusH_Pos = Key_and_Pos(E_minusH_minusH)
            E_minusH_minusH = PES(E_minusH_minusH_key, E_minusH_minusH_Pos, molecule)
            #E_minusH_H
            E_minusH_H = np.copy(Pos)
            E_minusH_H[0][i] = E_minusH_H[0][i] - (hessunit * unit_sort)
            E_minusH_H[0][j] = E_minusH_H[0][j] + (hessunit * unit_sort)
            E_minusH_H = E_minusH_H.reshape(dimension,3)
            E_minusH_H_key, E_minusH_H_Pos = Key_and_Pos(E_minusH_H)
            E_minusH_H = PES(E_minusH_H_key, E_minusH_H_Pos, molecule)
            #E_H_minusH
            E_H_minusH = np.copy(Pos)
            E_H_minusH[0][i] = E_H_minusH[0][i] + (hessunit * unit_sort)
            E_H_minusH[0][j] = E_H_minusH[0][j] - (hessunit * unit_sort)
            E_H_minusH = E_H_minusH.reshape(dimension,3)
            E_H_minusH_key, E_H_minusH_Pos = Key_and_Pos(E_H_minusH)
            E_H_minusH = PES(E_H_minusH_key, E_H_minusH_Pos, molecule)
            #E_H_H
            E_H_H = np.copy(Pos)
            E_H_H[0][i] = E_H_H[0][i] + (hessunit * unit_sort)
            E_H_H[0][j] = E_H_H[0][j] + (hessunit * unit_sort)
            E_H_H = E_H_H.reshape(dimension,3)
            E_H_H_key, E_H_H_Pos = Key_and_Pos(E_H_H)
            E_H_H = PES(E_H_H_key, E_H_H_Pos, molecule)
            
            #Eta_ij Full equ.
            eta_ij = (1 / (4 * (hessunit)**2)) * (E_minusH_minusH - E_minusH_H - E_H_minusH + E_H_H) 
            Hessian[i][j] = (eta_ij)

        elif i == j:
            #E_2minusH_0
            E_2minusH_0 = np.copy(Pos)
            E_2minusH_0[0][i] = E_2minusH_0[0][i] - (2 * hessunit * unit_sort)
            E_2minusH_0 = E_2minusH_0.reshape(dimension,3)
            E_2minusH_0_key, E_2minusH_0_Pos = Key_and_Pos(E_2minusH_0)
            E_2minusH_0 = PES(E_2minusH_0_key, E_2minusH_0_Pos, molecule)
            #E_minusH_0
            E_minusH_0 = np.copy(Pos)
            E_minusH_0[0][i] = E_minusH_0[0][i] - (hessunit * unit_sort)
            E_minusH_0 = E_minusH_0.reshape(dimension,3)
            E_minusH_0_key, E_minusH_0_Pos = Key_and_Pos(E_minusH_0)
            E_minusH_0 = PES(E_minusH_0_key, E_minusH_0_Pos, molecule)
            #E_0_0
            E_0_0 = np.copy(Pos)
            E_0_0 = E_0_0.reshape(dimension,3)
            E_0_0_key, E_0_0_Pos = Key_and_Pos(E_0_0)
            E_0_0 = PES(E_0_0_key, E_0_0_Pos, molecule)
            #E_H_0
            E_H_0 = np.copy(Pos)
            E_H_0[0][i] = E_H_0[0][i] + (hessunit * unit_sort)
            E_H_0 = E_H_0.reshape(dimension,3)
            E_H_0_key, E_H_0_Pos = Key_and_Pos(E_H_0)
            E_H_0 = PES(E_H_0_key, E_H_0_Pos, molecule)
            #E_2H_0
            E_2H_0 = np.copy(Pos)
            E_2H_0[0][i] = E_2H_0[0][i] + (2 * hessunit * unit_sort)
            E_2H_0 = E_2H_0.reshape(dimension,3)
            E_2H_0_key, E_2H_0_Pos = Key_and_Pos(E_2H_0)
            E_2H_0 = PES(E_2H_0_key, E_2H_0_Pos, molecule)
            
            #Eta_ii Full equ.
            eta_ii = (1 / (12 * (hessunit)**2)) * (-E_2minusH_0 + (16 * E_minusH_0) - (30 * E_0_0) + (16 * E_H_0) - E_2H_0)
            Hessian[i][i] = (eta_ii)
    
    # **** PRINT STATEMENTS ****
    #print("Hessian [Energy Unit][Bohr]^-2 =") 
    #print(Hessian)
    
    
    
    return Hessian

.

## [E.3] Mass weighting the Hessian 

Requires the input of a Hessian, the "Key" for a molecule and the mass index which needs to be created remotely.

Input masses need to be in:  Atomic masses [m$_e$].

Output: $\eta_{ii}^{MWC}$ = $\frac{\left[Hartrees \right]}{ \left[Bohrs\right]^2\left[m_e\right]}$

NOTE: $\omega$ = $\sqrt(\eta_{ii}^{MWC})$ $\therefore$ = $\frac{\left[Hartrees \right]^{1/2}}{ \left[Bohrs\right]\left[m_e\right]^{1/2}}$

In [6]:
def Hess_MWC(Hess_CART, Key, mass):
    
    dimension = int(Key[0])
    #print(dimension)
    
    i = np.arange(0, (3*dimension))
    j = np.arange(0, (3*dimension))
    #print(len(m))
    
    Hess_CART = np.copy(Hess_CART)
    Hess_MWC = np.zeros(((3*dimension), (3*dimension)))
    #Weight_matrix = np.zeros(((3*dimension), (3*dimension)))
    
    #Diatomic
    if len(mass) == 2:
        #print("Diatomic")
        mass = [mass[0], mass[0], mass[0], mass[1], mass[1], mass[1]] 
      
    #Triatomic

    elif len(mass) == 3:
        #print("Triatomic")
        mass = [mass[0], mass[0], mass[0], mass[1], mass[1], mass[1], mass[2], mass[2], mass[2]]
        #m = np.array(m)
        
    #Tetratomic 
    elif len(mass) == 4:
        mass = [mass[0], mass[0], mass[0], mass[1], mass[1], mass[1], mass[2], mass[2], mass[2], mass[3], mass[3], mass[3]]
        
    #Polyatomic
    elif len(mass) >= 5:
        print("Polyatomic not yet functional")
    
    for i,j in itertools.product(i, j):
            Hess_MWC[i][j] = Hess_CART[i][j] / (mass[i] * mass[j])**.5
            #Weight_matrix[i][j] = 1 / (mass[i] * mass[j])**.5
    
    # **** PRINT STATEMENTS ****
    #print("Mass Weighted Hessian [Energy Unit][Bohr]^{-2}[m_e]^{-1} =") 
    #print(Hess_MWC)
    #print("----------------")
    #print(significant_figures(Weight_matrix, 4))
    
    return Hess_MWC

.

## [E.4] Eigenvalues and Eigenvectors w/ Diagonalisation

Need to ensure you have "import scipy.linalg as la" at the start of the code.
Main Equation:

${\nu}$ = $\sqrt{\frac{\eta_{ii}}{4\pi^2c_{cm}^2}}$ where, $cm^{-1}$ = $\sqrt{\frac{[J] [m]^{-2} [kg]^{-1}}{[cm]^2 [s]^{-2}}}$

In [7]:
def eigval_eigvec(Hess_MWC):
    
    e_converter = hartree_to_j #Change as appropriate
    length_converter = bohr_to_m
    mass_converter = me_to_kg
    
    eigvals, eigvecs = la.eig(Hess_MWC)
    eigvals = eigvals.real
    Hess_diag_au = np.diag(eigvals)
    atomic = (np.copy(Hess_diag_au)) * (e_converter/(length_converter**2 * mass_converter))
    wavenumber = (abs(atomic)/(4 * np.pi**2 * c_cm**2))**0.5
    threshold_indices = wavenumber < 25 #Set values to 0 if less than 50 wavenumbers
    wavenumber[threshold_indices] = 0
    fundamentals_wn = wavenumber[np.where(wavenumber!=0)]
    fundamentals_au = wn_to_hartree * fundamentals_wn
        
    # **** PRINT STATEMENTS ****
    #print("Eigenvalues =", 
    #print(eigvals)
    #print("----------------------------------")
    #print("Eigenvectors =")
    #print(eigvecs)
    #print("----------------------------------")
    #print("Diagonlised Hessian [au] =")
    #print(Hess_diag_au)
    #print("----------------------------------")
    #print("Diagonlised Hessian [cm]^-1 =")
    #print(wavenumber)
    #print("----------------------------------")
    #print("Fundamental frequencies [au] =")
    #print(fundamentals_au)
    #print("----------------------------------")
    print("Fundamental frequencies [cm]^-1 =")
    print(fundamentals_wn)
    
    return eigvals, eigvecs, Hess_diag_au, wavenumber, fundamentals_au , fundamentals_wn

.

## [E.5] Un-mass-weighting the Eigenvectors

This it to return the eigenvectors to a "normal" format.

In [8]:
def UMW(eigvec, m, Key):
    
    dimension = int(Key[0])
    
    UMW_eigvec = np.zeros((3*dimension, 3*dimension))
    
    i = np.arange(0, 3*dimension)
    j = np.arange(0, 3*dimension)
    #print(i)
    
    #Diatomic
    if len(m) == 2:
        m = [m[0],m[0],m[0],m[1],m[1],m[1]] 
      
    #Triatomic
    elif len(m) == 3:
        m = [m[0],m[0],m[0],m[1],m[1],m[1],m[2],m[2],m[2]]
        #m = np.array(m)
        
    #Tetratomic 
    elif len(m) == 4:
        m = [m[0],m[0],m[0],m[1],m[1],m[1],m[2],m[2],m[2],m[3],m[3],m[3]]
        
    #Polyatomic
    elif len(m) >= 5:
        print("Polyatomic not yet functional")
    
    for i,j in itertools.product(i, j):
        UMW_eigvec[i][j] = eigvec[i][j] / (m[i])**.5
    
    # **** PRINT STATEMENTS ****
    #print("Eigenvectors UMW =")
    #print(UMW_eigvec)
    
    return UMW_eigvec

.

## [E.6] Calculating the displacement vectors and corresponding eigenvalues

This is the new name for the "normal modes", to remove confusion.

In [9]:
def displace_vec_and_val(eigvals, UMW_eigvec):
    
    #Initialise an empty matrix to insert "good" displacement vectors into
    displace_vec = []
    displace_val = []
    
    for i in range(len(eigvals)):
        if eigvals[i] >= 5e-6: 
            #Any eigval [au] below will be too small 
            #to be a realistic freq
            
            goodvec = list(UMW_eigvec[:,i])
            #print(eigvals[i], goodvec) 
            #print("-------------------------")
            displace_vec.append(goodvec)
            displace_val.append(eigvals[i])
        
    #print(isinstance(displace_vec, list))
    displace_vec = np.array(displace_vec)
    
    # **** PRINT STATEMENTS ****
    #print("Displace values [au] =", 
    #print(displace_val)
    #print("----------------------------------")
    #print("Displace vectors [au] =")
    #print(displace_vec)
    #print("----------------------------------")

    return displace_vec, displace_val

.

## [E.7] Calculating the cubic constants 

\begin{equation}
\label{eq:eta_iii}
    \eta_{iii} = \frac{1}{2h^3} \left( -E_{\left(-2,0\right)} + 2E_{\left(-1,0\right)} - 2E_{\left(1,0\right)} + E_{\left(2,0\right)} \right)
\end{equation}
\begin{equation}
\label{eq:eta_iij}
    \eta_{iij} = \frac{1}{2h^3} \left( - E_{\left(-1,-1\right)} + E_{\left(-1,1\right)} + 2E_{\left(0,-1\right)} - 2E_{\left(0,1\right)} - E_{\left(1,-1\right)} + E_{\left(1,1\right)} \right)
\end{equation}

These are automatically mass weighted thanks to the shifting step size parameter being taken using the frequencies.

Output:
$\eta_{iii}$ = $\frac{\left[Hartrees \right]}{\left[m_e\right]^{3/2} \left[Bohrs\right]^3}$

In [3]:
#Function to calculate the cubic dreivaties
def cubic_CART(Key, Pos, displace_vec, displace_val, PES, molecule):
    
    #Initialisation of Hessian and Dimensions
    #This work irregardless of molecule size
    maxmode = len(displace_vec)
    dimension = int(Key[0])
    #print(dimension)
    cubic = np.zeros((maxmode, maxmode))
    
    cubic_displace_vec = np.copy(displace_vec)
    
    i = np.arange(0, maxmode) #has to go from 0 to maxmode
    j = np.arange(0, maxmode) #has to go from 0 to maxmode
    #print(i)
    #print(j)
    
    Pos = np.copy(Pos)
    Pos = Pos.reshape(1, 3*dimension)
    
    for i,j in product(i, j):
        #print("i,j", i,j)
        
        omega_i = np.sqrt(displace_val[i])
        #print("omega_i", omega_i)
        step_size_i = (1 / np.sqrt(omega_i))
        #print("step_size_i", step_size_i)
        omega_j = np.sqrt(displace_val[j])
        #print("omega_j", omega_j)
        step_size_j = (1 / np.sqrt(omega_j))
        #print("step_size_j", step_size_j)
        #print("-----------------------")

        unit_sort = bohr_to_angstrom
        
        #eta_iij
        if i != j:
            #E_minusH_minusH (A)
            temp_pos_A = np.copy(Pos)
            temp_pos_A = temp_pos_A + (-1 * step_size_i * unit_sort * np.array(cubic_displace_vec[i][:])) + (-1 * step_size_j * unit_sort * np.array(cubic_displace_vec[j][:]))
            temp_pos_A = temp_pos_A.reshape(dimension, 3)
            E_minusH_minusH_key, E_minusH_minusH_Pos = Key_and_Pos(temp_pos_A)
            E_minusH_minusH = PES(E_minusH_minusH_key, E_minusH_minusH_Pos, molecule)
            #print("A", E_minusH_minusH)
            #E_minusH_H (B)
            temp_pos_B = np.copy(Pos) 
            temp_pos_B = temp_pos_B + (-1 * step_size_i * unit_sort * np.array(cubic_displace_vec[i][:])) + (1 * step_size_j * unit_sort * np.array(cubic_displace_vec[j][:]))
            temp_pos_B = temp_pos_B.reshape(dimension, 3)
            E_minusH_H_key, E_minusH_H_Pos = Key_and_Pos(temp_pos_B)
            E_minusH_H = PES(E_minusH_H_key, E_minusH_H_Pos, molecule)
            #print("B", E_minusH_H)
            #E_0_minusH (C)
            temp_pos_C = np.copy(Pos) 
            temp_pos_C = temp_pos_C + (-1 * step_size_j * unit_sort * np.array(cubic_displace_vec[j][:]))
            temp_pos_C = temp_pos_C.reshape(dimension, 3)
            E_0_minusH_key, E_0_minusH_Pos = Key_and_Pos(temp_pos_C)
            E_0_minusH = PES(E_0_minusH_key, E_0_minusH_Pos, molecule)
            #print("C", E_0_minusH)
            #E_0_H (D)
            temp_pos_D = np.copy(Pos) 
            temp_pos_D = temp_pos_D + (1 * step_size_j * unit_sort * np.array(cubic_displace_vec[j][:]))
            temp_pos_D = temp_pos_D.reshape(dimension, 3)
            E_0_H_key, E_0_H_Pos = Key_and_Pos(temp_pos_D)
            E_0_H = PES(E_0_H_key, E_0_H_Pos, molecule)
            #print("D", E_0_H)
            #E_H_minusH (E)
            temp_pos_E = np.copy(Pos)
            temp_pos_E = temp_pos_E + (1 * step_size_i * unit_sort * np.array(cubic_displace_vec[i][:])) + (-1 * step_size_j * unit_sort * np.array(cubic_displace_vec[j][:]))
            temp_pos_E = temp_pos_E.reshape(dimension, 3)
            E_H_minusH_key, E_H_minusH_Pos = Key_and_Pos(temp_pos_E)
            E_H_minusH = PES(E_H_minusH_key, E_H_minusH_Pos, molecule)
            #print("E", E_H_minusH)
            #E_H_H (F)
            temp_pos_F = np.copy(Pos) 
            temp_pos_F = temp_pos_F + (1 * step_size_i * unit_sort * np.array(cubic_displace_vec[i][:])) + (1 * step_size_j * unit_sort * np.array(cubic_displace_vec[j][:]))
            temp_pos_F = temp_pos_F.reshape(dimension, 3)
            E_H_H_key, E_H_H_Pos = Key_and_Pos(temp_pos_F)
            E_H_H = PES(E_H_H_key, E_H_H_Pos, molecule)
            #print("F", E_H_H)
            
            #Eta_iij Full equ
            eta_iij = (1 / (2 * (step_size_i**2 * step_size_j))) * (-E_minusH_minusH + E_minusH_H + (2 * E_0_minusH) - (2 * E_0_H) - E_H_minusH + E_H_H) 
            #print(eta_iij)
            cubic[i][j] = (eta_iij)
            #print("ij",i,j, "cubic val.", eta_iij)

        #eta_iii
        elif i == j:
            #E_2minusH_0 (G)
            temp_pos_G = np.copy(Pos) 
            temp_pos_G = temp_pos_G + (-2 * step_size_i * unit_sort * np.array(cubic_displace_vec[i][:]))
            temp_pos_G = temp_pos_G.reshape(dimension, 3)
            E_2minusH_0_key, E_2minusH_0_Pos = Key_and_Pos(temp_pos_G)
            E_2minusH_0 = PES(E_2minusH_0_key, E_2minusH_0_Pos, molecule)
            #print("G", E_2minusH_0)
            #E_minusH_0 (H)
            temp_pos_H = np.copy(Pos) 
            temp_pos_H = temp_pos_H + (-1 * step_size_i * unit_sort * np.array(cubic_displace_vec[i][:]))
            temp_pos_H = temp_pos_H.reshape(dimension, 3)
            E_minusH_0_key, E_minusH_0_Pos = Key_and_Pos(temp_pos_H)
            E_minusH_0 = PES(E_minusH_0_key, E_minusH_0_Pos, molecule)
            #print("H", E_minusH_0)
            #E_H_0 (I)
            temp_pos_I = np.copy(Pos) 
            temp_pos_I = temp_pos_I + (1 * step_size_i * unit_sort * np.array(cubic_displace_vec[i][:]))
            temp_pos_I = temp_pos_I.reshape(dimension, 3)
            E_H_0_key, E_H_0_Pos = Key_and_Pos(temp_pos_I)
            E_H_0 = PES(E_H_0_key, E_H_0_Pos, molecule)
            #print("I", E_H_0)
            #E_2H_0 (J)
            temp_pos_J = np.copy(Pos)
            temp_pos_J = temp_pos_J + (2 * step_size_i * unit_sort * np.array(cubic_displace_vec[i][:]))
            temp_pos_J = temp_pos_J.reshape(dimension, 3)
            E_2H_0_key, E_2H_0_Pos = Key_and_Pos(temp_pos_J)
            E_2H_0 = PES(E_2H_0_key, E_2H_0_Pos, molecule)
            #print("J", E_2H_0)
            
            #Eta_iii Full equ
            eta_iii = (1 / (2 * (step_size_i**3))) * (-E_2minusH_0 + (2 * E_minusH_0) - (2 * E_H_0) + E_2H_0)
            #print(eta_iii)
            cubic[i][i] = (eta_iii)
            #print("ij",i,i, "cubic val.", eta_iii)
    
    # **** PRINT STATEMENTS ****
    #print("Cubic constants [au] =")
    #print(cubic)
    return cubic

.

## [E.8] Calculating the quartic constants  

$\eta_{iiii}$ = $\frac{1}{h^4} \left(E_{\left(-2,0\right)} - 4E_{\left(-1,0\right)} + 6E_{\left(0,0\right)} - 4E_{\left(1,0\right)} + E_{\left(2,0\right)} \right)$

$\eta_{iijj}$ = $\frac{1}{h^4} ( E_{\left(-1,-1\right)} - 2E_{\left(-1,0\right)} + E_{\left(-1,1\right)} - 2E_{\left(0,-1\right)} + 4E_{\left(0,0\right)} - 2E_{\left(0,1\right)} + E_{\left(1,-1\right)} - 2E_{\left(1,0\right)} + E_{\left(1,1\right)} )$

These are automatically mass weighted thanks to the shifting step size parameter being taken using the frequencies.

Output:
$\eta_{iiii}$ = $\frac{\left[Hartrees \right]}{\left[m_e\right]^{2} \left[Bohrs\right]^4}$

In [11]:
#Function to calculate the Cartesian Hessian
def quartic_CART(Key, Pos, displace_vec, displace_val, PES, molecule):
    
    #Initialisation of Hessian and Dimensions
    #This work irregardless of molecule size
    maxmode = len(displace_vec)
    dimension = int(Key[0])
    #print(dimension)
    quartic = np.zeros((maxmode, maxmode))
    
    quartic_displace_vec = np.copy(displace_vec)
    
    i = np.arange(0, maxmode) #has to go from 0 to maxmode
    j = np.arange(0, maxmode) #has to go from 0 to maxmode
    
    Pos = np.copy(Pos)
    Pos = Pos.reshape(1, 3*dimension)
    
    for i,j in product(i, j):
        
        #print("i,j", i,j)
        omega_i = np.sqrt(displace_val[i])
        #print("omega_i", omega_i)
        step_size_i = (1 / np.sqrt(omega_i))
        #print("step_size_i", step_size_i)
        omega_j = np.sqrt(displace_val[j])
        #print("omega_j", omega_j)
        step_size_j = (1 / np.sqrt(omega_j))
        #print("step_size_j", step_size_j)
        #print("-----------------------")
        
        unit_sort = bohr_to_angstrom
      
        #eta_iijj
        if i != j:
            #E_minusH_minusH
            temp_pos_A = np.copy(Pos)
            temp_pos_A = temp_pos_A + (-1 * step_size_i * unit_sort * np.array(quartic_displace_vec[i][:])) + (-1 * step_size_j * unit_sort * np.array(quartic_displace_vec[j][:]))
            temp_pos_A = temp_pos_A.reshape(dimension, 3)
            E_minusH_minusH_key, E_minusH_minusH_Pos = Key_and_Pos(temp_pos_A)
            E_minusH_minusH = PES(E_minusH_minusH_key, E_minusH_minusH_Pos, molecule)
            #E_minusH_0
            temp_pos_B = np.copy(Pos) 
            temp_pos_B = temp_pos_B + (-1 * step_size_i * unit_sort * np.array(quartic_displace_vec[i][:])) 
            temp_pos_B = temp_pos_B.reshape(dimension, 3)
            E_minusH_0_key, E_minusH_0_Pos = Key_and_Pos(temp_pos_B)
            E_minusH_0 = PES(E_minusH_0_key, E_minusH_0_Pos, molecule)
            #E_minusH_H
            temp_pos_C = np.copy(Pos) 
            temp_pos_C = temp_pos_C + (-1 * step_size_i * unit_sort * np.array(quartic_displace_vec[i][:])) + (1 * step_size_j * unit_sort * np.array(quartic_displace_vec[j][:])) 
            temp_pos_C = temp_pos_C.reshape(dimension, 3)
            E_minusH_H_key, E_minusH_H_Pos = Key_and_Pos(temp_pos_C)
            E_minusH_H = PES(E_minusH_H_key, E_minusH_H_Pos, molecule)
            #E_0_minusH
            temp_pos_D = np.copy(Pos) 
            temp_pos_D = temp_pos_D + (-1 * step_size_j * unit_sort * np.array(quartic_displace_vec[j][:])) 
            temp_pos_D = temp_pos_D.reshape(dimension, 3)
            E_0_minusH_key, E_0_minusH_Pos = Key_and_Pos(temp_pos_D)
            E_0_minusH = PES(E_0_minusH_key, E_0_minusH_Pos, molecule)
            #E_0_0
            temp_pos_E = np.copy(Pos)
            temp_pos_E = temp_pos_E.reshape(dimension, 3)
            E_0_0_key, E_0_0_Pos = Key_and_Pos(temp_pos_E)
            E_0_0 = PES(E_0_0_key, E_0_0_Pos, molecule)
            #E_0_H
            temp_pos_F = np.copy(Pos) 
            temp_pos_F = temp_pos_F + (1 * step_size_j * unit_sort * np.array(quartic_displace_vec[j][:])) 
            temp_pos_F = temp_pos_F.reshape(dimension, 3)
            E_0_H_key, E_0_H_Pos = Key_and_Pos(temp_pos_F)
            E_0_H = PES(E_0_H_key, E_0_H_Pos, molecule)
            #E_H_minusH
            temp_pos_G = np.copy(Pos) 
            temp_pos_G = temp_pos_G + (1 * step_size_i * unit_sort * np.array(quartic_displace_vec[i][:])) + (-1 * step_size_j * unit_sort * np.array(quartic_displace_vec[j][:])) 
            temp_pos_G = temp_pos_G.reshape(dimension, 3)
            E_H_minusH_key, E_H_minusH_Pos = Key_and_Pos(temp_pos_G)
            E_H_minusH = PES(E_H_minusH_key, E_H_minusH_Pos, molecule)
            #E_H_0
            temp_pos_H = np.copy(Pos)
            temp_pos_H = temp_pos_H + (1 * step_size_i * unit_sort * np.array(quartic_displace_vec[i][:])) 
            temp_pos_H = temp_pos_H.reshape(dimension, 3)
            E_H_0_key, E_H_0_Pos = Key_and_Pos(temp_pos_H)
            E_H_0 = PES(E_H_0_key, E_H_0_Pos, molecule)
            #E_H_H
            temp_pos_I = np.copy(Pos) 
            temp_pos_I = temp_pos_I + (1 * step_size_i * unit_sort * np.array(quartic_displace_vec[i][:])) + (1 * step_size_j * unit_sort * np.array(quartic_displace_vec[j][:])) 
            temp_pos_I = temp_pos_I.reshape(dimension, 3)
            E_H_H_key, E_H_H_Pos = Key_and_Pos(temp_pos_I)
            E_H_H = PES(E_H_H_key, E_H_H_Pos, molecule)
            
            #eta_iijj Full equ
            eta_iijj = (1 / ((step_size_i)**2 * (step_size_j)**2)) * (E_minusH_minusH - (2 * E_minusH_0) + E_minusH_H - (2 * E_0_minusH) + (4 * E_0_0) - (2 * E_0_H) + E_H_minusH - (2 * E_H_0) + E_H_H) 
            quartic[i][j] = (eta_iijj)
            #print("iijj", i,i,j,j, " === ", eta_iijj)
            #print(quartic)
            
        elif i == j:
            #E_2minusH_0
            temp_pos_J = np.copy(Pos) 
            temp_pos_J = temp_pos_J + (-2 * step_size_i * unit_sort * np.array(quartic_displace_vec[i][:]))
            temp_pos_J = temp_pos_J.reshape(dimension, 3)
            E_2minusH_0_key, E_2minusH_0_Pos = Key_and_Pos(temp_pos_J)
            E_2minusH_0 = PES(E_2minusH_0_key, E_2minusH_0_Pos, molecule)
            #E_minusH_0
            temp_pos_K = np.copy(Pos) 
            temp_pos_K = temp_pos_K + (-1 * step_size_i * unit_sort * np.array(quartic_displace_vec[i][:]))
            temp_pos_K = temp_pos_K.reshape(dimension, 3)
            E_minusH_0_key, E_minusH_0_Pos = Key_and_Pos(temp_pos_K)
            E_minusH_0 = PES(E_minusH_0_key, E_minusH_0_Pos, molecule)
            #E_0_0
            temp_pos_L = np.copy(Pos) 
            temp_pos_L = temp_pos_L.reshape(dimension, 3)
            E_0_0_key, E_0_0_Pos = Key_and_Pos(temp_pos_L)
            E_0_0 = PES(E_0_0_key, E_0_0_Pos, molecule)
            #E_H_0
            temp_pos_M = np.copy(Pos) 
            temp_pos_M = temp_pos_M + (1 * step_size_i * unit_sort * np.array(quartic_displace_vec[i][:]))
            temp_pos_M = temp_pos_M.reshape(dimension, 3)
            E_H_0_key, E_H_0_Pos = Key_and_Pos(temp_pos_M)
            E_H_0 = PES(E_H_0_key, E_H_0_Pos, molecule)
            #E_2H_0
            temp_pos_N = np.copy(Pos)
            temp_pos_N = temp_pos_N + (2 * step_size_i * unit_sort * np.array(quartic_displace_vec[i][:]))
            temp_pos_N = temp_pos_N.reshape(dimension, 3)
            E_2H_0_key, E_2H_0_Pos = Key_and_Pos(temp_pos_N)
            E_2H_0 = PES(E_2H_0_key, E_2H_0_Pos, molecule)
            
            #Eta_iiii Full equ
            eta_iiii = (1 / (step_size_i)**4) * (E_2minusH_0 - (4 * E_minusH_0) + (6 * E_0_0) - (4 * E_H_0) + E_2H_0)
            quartic[i][i] = (eta_iiii)
            #print("iijj", i,i,j,j, " === ", eta_iiii)
            #print(quartic)
        
    # **** PRINT STATEMENTS ****
    #print("Quartic constants [au] =")
    #print(quartic)
    
    return quartic

.

## [E.9] Calculating Sigma, $\sigma_{ij}$ 

$\sigma_{ij}$ = $\frac{\left(\delta_{ij} - 2\right)\left(\omega_i + \omega_j\right)\eta_{iij}}{4\omega_i \omega_j^2 \left(2\omega_i + \omega_j \right)} - \sum_k \frac{\eta_{kkj}}{4\omega_k \omega_j^2}$

Sigma has a strong dependence on the calculation of the cubic constants.

Output:
$\sigma_{ij}$ = $\frac{\frac{\left[Hartrees \right]^{3/2}}{ \left[Bohrs\right]^4\left[m_e\right]^{2}}}{\frac{\left[Hartrees \right]^{2}}{ \left[Bohrs\right]^4\left[m_e\right]^{2}}} - \sum_k \frac{\frac{\left[Hartrees \right]}{\left[Bohrs\right]^3\left[m_e\right]^{3/2}}}{\frac{\left[Hartrees \right]^{3/2}}{ \left[Bohrs\right]^3\left[m_e\right]^{3/2}}}$ = $\frac{1}{[Hartrees]^{1/2}}$

However thanks the atomic $\hbar$ with units $\left[Hartrees\right]\left[as\right]$, we can convert as follows:
$\sigma_{ij}$ = $\frac{\left[Hartrees\right]\left[as\right]}{[Hartrees]^{1/2}}$ = $\left[Bohrs\right]\left[m_e\right]^{1/2}$



In [12]:
def sigma_ij(modes, cubic):
    
    maxmode = len(modes)
    
    modes = np.sqrt(modes)
    
    sigma = np.zeros((maxmode, maxmode))
    
    i = np.arange(0, maxmode) #has to go from 0 to maxmode
    j = np.arange(0, maxmode) #has to go from 0 to maxmode
    
    for i,j in product(i, j):
        
        if i == j:
            kdelta = 1
        else:
            kdelta = 0
        
        #print("for", i,j, "kdelta =", kdelta)
        #print("i =", i)
        #print("j (loop1)", j)
        
        k = np.arange(0, maxmode) #has to go from 0 to maxmode
        
        #print("---------------")
        num = (kdelta - 2) * (modes[i] + modes[j]) * cubic[i][j]
        denom = 4 * modes[i] * modes[j]**2 * ((2 * modes[i]) + modes[j])
        partA = num / denom
        #print("part A", partA)
        #print("---------------")
        
        summ = 0
        for k in k:
            #print("---------------")
            #print("SUM PART")
            #print("k (in loop2)", k)
            #print("j (in loop2)", j)
            sumtemp = (cubic[k][j])/(4 * modes[k] * modes[j]**2)
            #print("cubic[k][j]", cubic[k][j])
            #print("modes[k]", modes[k])
            #print("modes[j]", modes[j])
            #print("sumtemp =", sumtemp)
            summ += sumtemp           
        #print("Sum =", summ)
        #print("---------------------------------")
        
        sigma_temp = partA - summ
        #print("Total = ", sigma_temp)
        #print("---------------------------------")
        
        #print("ij", i,j, "sigma =", sigma_temp)
        sigma[i][j] = (sigma_temp)
        
        #print("---------------------------------")
        
        # **** PRINT STATEMENTS ****
        #print("Sigma constants [au] =")
        #print(sigma)

    return sigma

.

## [E.10] Calculating deltaETOSH, $\Delta E_{ij}^{TOSH}$ 

$\Delta E_i^{TOSH}$ = $\omega_i + \frac{1}{8\omega_i} \sum_j \frac{\eta_{iijj}}{\omega_j} + \frac{1}{2\omega_i} \sum_j \eta_{iij}\sigma_{ij} + \frac{1}{4\omega_i} \sum_{j,k} \eta_{iijk}\sigma_{ij}\sigma_{ik}$

....

$\Delta E_i^{TOSH}$ = $\frac{\left[Hartrees \right]^{1/2}}{ \left[Bohrs\right]\left[m_e\right]^{1/2}} + \frac{1}{\frac{\left[Hartrees \right]^{1/2}}{ \left[Bohrs\right]\left[m_e\right]^{1/2}}} \frac{\frac{\left[Hartrees \right]}{\left[m_e\right]^{2} \left[Bohrs\right]^4}}{\frac{\left[Hartrees \right]^{1/2}}{ \left[Bohrs\right]\left[m_e\right]^{1/2}}} + \frac{1}{\frac{\left[Hartrees \right]^{1/2}}{ \left[Bohrs\right]\left[m_e\right]^{1/2}}}  \frac{\left[Hartrees \right]}{\left[m_e\right]^{3/2} \left[Bohrs\right]^3}\frac{1}{[Hartrees]^{1/2}} + \frac{1}{\frac{\left[Hartrees \right]^{1/2}}{ \left[Bohrs\right]\left[m_e\right]^{1/2}}} \frac{\left[Hartrees \right]}{\left[m_e\right]^{2} \left[Bohrs\right]^4}\frac{1}{[Hartrees]^{1/2}}\frac{1}{[Hartrees]^{1/2}}$

$\therefore$

$\Delta E_i^{TOSH}$ = $\frac{\left[Hartrees \right]^{1/2}}{ \left[Bohrs\right]\left[m_e\right]^{1/2}} + \frac{1}{\left[Bohrs\right]^2\left[m_e\right]} + \frac{1}{\left[Bohrs\right]^2\left[m_e\right]} + \frac{1}{\left[Hartrees \right]^{1/2}\left[Bohrs\right]^3\left[m_e\right]^{3/2}}$

However thanks the atomic $\hbar$ with units $\left[Hartrees\right]\left[as\right]$, we can convert as follows:

$\Delta E_i^{TOSH}$ = $\omega_i\left(\hbar\right) + \frac{1}{8\omega_i} \sum_j \frac{\eta_{iijj}}{\omega_j}\left(\hbar\right)^2 + \frac{1}{2\omega_i} \sum_j \eta_{iij}\sigma_{ij}\left(\hbar\right)^2 + \frac{1}{4\omega_i} \sum_{j,k} \eta_{iijk}\sigma_{ij}\sigma_{ik}\left(\hbar\right)^3$

$\Delta E_i^{TOSH}$ = $\left[Hartrees\right]$

In [7]:
def deltaETOSH_ij(modes, cubic, quartic, sigma):
    #print(sigma)
    if len(str(sigma)) == 1:
        #print("Harmonic")
        modes = np.array(modes)
        omega = np.sqrt(modes)
        deltaETOSH = omega * hartree_to_wn
    else:    
        #print("Anharmonic")
        #Need input in purely atomic units

        maxmode = len(modes)
        #print(maxmode)

        modes = np.array(modes)
        omega = np.sqrt(modes)

        i = np.arange(0, maxmode) #has to go from 0 to maxmode
        deltaETOSH = np.zeros((maxmode, 1))
        
        for i in i:
        
            part_A = omega[i]
            #print("part A =", part_A*hartree_to_wn)

            j = np.arange(0, maxmode) #has to go from 0 to maxmode
            j = np.array(j)

            sumtemp_B = 0
            sumtemp_C = 0
            sumtemp_D = 0

            if len(j) != 1:
            #Summation Loop
                for j in j:
                    #print("loop", j)

                    temp_B = (quartic[i][j]) / (omega[j])
                    #print(temp_B)
                    sumtemp_B += temp_B
                    #print(sumtemp_B)

                    temp_C = (cubic[i][j]) * (sigma[i][j])
                    #print(temp_C)
                    sumtemp_C += temp_C
                    #print(sumtemp_C)
                    
                    temp_D = (quartic[i][j]) * (sigma[i][j]) * (sigma[i][j])
                    #print(temp_D)   
                    sumtemp_D += temp_D
                    #print(sumtemp_D)
                
                #print("eta_iijj/omega_i contri.", sumtemp_B*hartree_to_wn)
                #print("eta_iij*sigma_ij contri.", sumtemp_C*hartree_to_wn)
                #print("eta_iijj contri.", sumtemp_D*hartree_to_wn)
                #print("eta_iijj*sigma_ij^2 contri.", sumtemp_D*hartree_to_wn)
                #print("------------")
                    
                    #k = np.arange(0, maxmode)
                    #print(k)
                    
                    #for k in k:
                        #temp_D = (quartic[i][j]) * (sigma[i][j]) * (sigma[i][k])
                        #print("temp",temp_D)
                        #sumtemp_D += temp_D
                        #print("sum", sumtemp_D)

            elif len(j) == 1:
                for j in j:
                    temp_B = (quartic[j]) / (omega[j])
                    #print(temp_B)
                    temp_C = (cubic[j]) * (sigma[j])
                    #print(temp_C)
                    temp_D = (quartic[j]) * np.square(sigma[j])
                    #print(temp_D)

                    #SUM
                    sumtemp_B += temp_B
                    #print(sumtemp_B)
                    sumtemp_C += temp_C
                    #print(sumtemp_C)
                    sumtemp_D += temp_D
                    #print(sumtemp_D)

            part_B = (1 / (8 * omega[i])) * sumtemp_B
            #print("part B =", part_B*hartree_to_wn)
            part_C = (1 / (2 * omega[i])) * sumtemp_C
            #print("part C =", part_C*hartree_to_wn)
            part_D = (1 / (4 * omega[i])) * sumtemp_D
            #print("part D =", part_D*hartree_to_wn)
            #print("------------")

            #print("--------------------")


            deltaETOSH[i] = part_A + part_B + part_C + part_D
            #print(deltaETOSH)
            #print("--------------------")

        deltaETOSH = (np.copy(deltaETOSH)) * hartree_to_wn

        if maxmode == 1:
            print("Corrected modes =", deltaETOSH, "vs Harm. =", omega*hartree_to_wn)
        elif maxmode == 3:
            print("Corrected modes =", deltaETOSH[0], "vs Harm. =", omega[0]*hartree_to_wn)
            print("Corrected modes =", deltaETOSH[1], "vs Harm. =", omega[1]*hartree_to_wn)
            print("Corrected modes =", deltaETOSH[2], "vs Harm. =", omega[2]*hartree_to_wn)
        elif maxmode == 4:
            print("Corrected modes =", deltaETOSH[0], "vs Harm. =", omega[0]*hartree_to_wn)
            print("Corrected modes =", deltaETOSH[1], "vs Harm. =", omega[1]*hartree_to_wn)
            print("Corrected modes =", deltaETOSH[2], "vs Harm. =", omega[2]*hartree_to_wn)
            print("Corrected modes =", deltaETOSH[3], "vs Harm. =", omega[3]*hartree_to_wn)
        else:
            print("This is not correct for a triatomic")

        # **** PRINT STATEMENTS ****
        #print("Corrected Band Origins =")
        #print(deltaETOSH)
    
    return deltaETOSH

.

## [E.11] Triatomic Coordinate Calculator

Input needs to be in angstroms and degrees (not rad yet)

In [14]:
def coord_triatomic(angle, distance_12, distance_23):
    
    #Treat central atom as the "origin"
    #Technically doesnt matter how its organised
    if angle == 180:
        m1 = [-distance_12, 0, 0]
        
        m2 = [0, 0, 0]
        
        m3 = [+distance_23, 0, 0]
        
    elif angle >= 90:
        newang = m.radians(90 - (angle/2))
        
        m1 = [(-np.cos(newang) * distance_12), (np.sin(newang) * distance_12), 0]
        
        m2 = [0,0,0]

        m3 = [(np.cos(newang) * distance_23), (np.sin(newang) * distance_23), 0]

    if angle < 90:
        print("Hmmm")
    
    coord = [m1, m2, m3]
    
    # **** PRINT STATEMENTS ****
    #print("Triatomic Coordinates =")
    #print(np.row_stack(coord))
    #print(coord)
    
    return coord

.

# [E.12] Rotational Constants, Inertia Momenets and COM Coordinates

\begin{equation}
\label{eq:Inertia_matrix}
    \textbf{I} =  
\begin{bmatrix}
I_{xx} & I_{xy} & I_{xz}
\\[6pt]
I_{yx} & I_{yy} & I_{yz}
\\[6pt]
I_{zx} & I_{zy} & I_{zz}
\end{bmatrix}
\end{equation}

Where: 
\begin{equation}
\label{eq:I_xx}
    I_{xx} = \sum_i m_i \left(y_i^2 + z_i^2\right)
\end{equation}
\begin{equation}
\label{eq:I_yy}
    I_{yy} = \sum_i m_i \left(x_i^2 + z_i^2\right)
\end{equation}
\begin{equation}
\label{eq:I_zz}
    I_{zz} = \sum_i m_i \left(x_i^2 + y_i^2\right)
\end{equation}
\begin{equation}
\label{eq:I_xy_yx}
    I_{xy} = I_{yx} = -\sum_i m_i x_i y_i
\end{equation}
\begin{equation}
\label{eq:I_xz_zx}
    I_{xz} = I_{zx} = -\sum_i m_i z_i y_i
\end{equation}
\begin{equation}
\label{eq:I_yz_zy}
    I_{yz} = I_{zy} = -\sum_i m_i y_i z_i
\end{equation}

Initial Unit: $I$ = $\left[Å\right]^2\left[m_e\right]$
Converted Units: $I$ = $\left[m\right]^2\left[kg\right]$

Rotational Constant Equation is:
\begin{equation}
    B = \frac{\hbar}{4\pi c_cm I_{eigenvalue}} = \frac{\left[J\right]\left[s\right]}{\left[cm\right]\left[s\right]^{-1}\left[m\right]^2\left[kg\right]} = \frac{\left[kg\right]\left[m\right]^2\left[s\right]^{-2}\left[s\right]}{\left[cm\right]\left[s\right]^{-1}\left[m\right]^2\left[kg\right]} = \frac{1}{\left[cm\right]}
\label{eq:rotational_constant}
\end{equation}

In [5]:
def rot_inertia_rcom(coords, masses, is_it_COMed):
    
    if is_it_COMed == "no":
        x_alpha = [element[0] for element in coords]
        y_alpha = [element[1] for element in coords]
        z_alpha = [element[2] for element in coords]

        #print("base", coords)
        #print("x", x_alpha)
        #print("y", y_alpha)
        #print("z", z_alpha)

        m_alpha__r_alpha__x = np.array(x_alpha) * np.array(masses)
        m_alpha__r_alpha__y = np.array(y_alpha) * np.array(masses)
        m_alpha__r_alpha__z = np.array(z_alpha) * np.array(masses)

        sum_mass = sum(masses)
        sum_m_alpha__r_alpha__x = sum(m_alpha__r_alpha__x)
        sum_m_alpha__r_alpha__y = sum(m_alpha__r_alpha__y)
        sum_m_alpha__r_alpha__z = sum(m_alpha__r_alpha__z)

        R_com_x = sum_m_alpha__r_alpha__x / sum_mass
        R_com_y = sum_m_alpha__r_alpha__y / sum_mass
        R_com_z = sum_m_alpha__r_alpha__z / sum_mass

        R_com = [R_com_x, R_com_y, R_com_z]
        R_com = np.array(R_com)

        #print("R_com", R_com)

        for i in np.arange(0, len(coords)):
            coords = np.array(coords)
            coords[i] = coords[i] - R_com

        #print("r_COMalpha", coords)
    
    else:
        coords = coords
        
    x_alpha = [element[0] for element in coords]
    y_alpha = [element[1] for element in coords]
    z_alpha = [element[2] for element in coords]
    
    #Inertia Matrix
    I_xx_temp = (np.array(np.square(y_alpha)) + np.array(np.square(z_alpha))) * np.array(masses)
    I_xx = sum(I_xx_temp)
    #print("I_xx", I_xx)

    I_xy_temp = (np.array(x_alpha) * np.array(y_alpha)) * np.array(masses)
    I_xy = -sum(I_xy_temp)
    #print("I_xy", I_xy)

    I_xz_temp = (np.array(x_alpha) * np.array(z_alpha)) * np.array(masses)
    I_xz = -sum(I_xz_temp)
    #print("I_xz", I_xz)

    I_yx_temp = (np.array(y_alpha) * np.array(x_alpha)) * np.array(masses)
    I_yx = -sum(I_yx_temp)
    #print("I_yx", I_yx)

    I_yy_temp = (np.array(np.square(x_alpha)) + np.array(np.square(z_alpha))) * np.array(masses)
    I_yy = sum(I_yy_temp)
    #print("I_yy", I_yy)

    I_yz_temp = (np.array(y_alpha) * np.array(z_alpha)) * np.array(masses)
    I_yz = -sum(I_yz_temp)
    #print("I_yz", I_yz)

    I_zx_temp = (np.array(z_alpha) * np.array(x_alpha)) * np.array(masses)
    I_zx = -sum(I_zx_temp)
    #print("I_zx", I_zx)

    I_zy_temp = (np.array(z_alpha) * np.array(y_alpha)) * np.array(masses)
    I_zy = -sum(I_zy_temp)
    #print("I_zy", I_zy)

    I_zz_temp = (np.array(np.square(x_alpha)) + np.array(np.square(y_alpha))) * np.array(masses)
    I_zz = sum(I_zz_temp)
    #print("I_zz", I_zz)

    I_prime = np.array([I_xx, I_xy, I_xz, I_yx, I_yy, I_yz, I_zx, I_zy, I_zz]).reshape((3, 3))
    #print(I_prime)

    eigvals_I, eigvecs = la.eig(I_prime)
    eigvals_I = eigvals_I.real
    #print("eigvals_I", eigvals_I)

    inertia_SI = eigvals_I * ((angstrom_to_bohr * bohr_to_m)**2 * me_to_kg)
    #print("inertia_SI", inertia_SI)
    
    np.seterr(divide='ignore', invalid='ignore')
    B = hbar / (c_cm * 4 * np.pi * inertia_SI)
    #print("Rot. Constants", B)
    
    B[B > 1e308] = 0
    
    B.sort()
    B = B[::-1]
    
    #print("B =", B)
    #print("I =", I)
    #print("coords =", coords)
        
    return B, eigvals_I, coords

.

## [E.13] All rotational constants calculator 

Creates a mode selection array of non-zero diagonal components:

\begin{equation}
\label{eq:mode_select_matrix}
    \textbf{Mode Selection} =  
\begin{bmatrix}
1 & 0 & 0 & \cdots & 0
\\[3pt]
0 & 1 & 0 & \cdots & 0
\\[3pt]
\vdots & \vdots & \vdots & \ddots & \vdots
\\[3pt]
0 & 0 & 0 & \cdots & 1
\end{bmatrix}
\end{equation}

The sigma array sorted by modes is then formed, for example for mode 1, we take the first slice to obtain the sigma for that mode:
\begin{equation}
\sigma_{n\times n} 
\begin{bmatrix}
1 
\\[3pt]
0 
\\[3pt]
\vdots 
\\[3pt]
0
\end{bmatrix}
 = 
\begin{bmatrix}
\sigma_{1, mode_1}
\\[3pt]
\sigma_{2, mode_1} 
\\[3pt]
\vdots 
\\[3pt]
\sigma_{n, mode_1}
\end{bmatrix}
\end{equation}

Now we can create the sigma sum array. This is done by by summing the eigenvector ($Q_n$) multiplied by the modal sigmas:

\begin{equation}
\sigma_{sum, mode_n} = \sigma_{1, mode_n} Q_1 + \sigma_{2, mode_n} Q_2 + \cdots + \sigma_{n, mode_n} Q_n
\end{equation}

This is then added to the equillibirum geometry:

\begin{equation}
X_{mode_n} = X_0 + \sigma_{sum, mode_n}
\end{equation}

We can then do the typical equations for each mode to get the new rotational constants.

In [33]:
def all_rot_const(eigvecs, sigma, coord, mass, unit_sort, COM):
    # Required constants:
    # bohr_to_angstrom & angstrom_to_bohr
    # kg_to_me & me_to_kg
    # bohr_to_m
    # from scipy.constants import h, c, hbar, u, k 
    #print(eigvecs)
    
    if len(str(sigma)) == 1:
        all_rot_const = "Harmonic"
    else:
        # Construcing the mode arrays
        length = len(eigvecs)
        #print("len eigvecs", length)

        #Constructing the mode selection arrays
        mode_arr = np.zeros((length, length))
        for i in range(0, length):
            mode_arr[i, i] = 1
        #print("mode_arr")
        #print(mode_arr)
        #print("--------------------")
        
        #print("sigma")
        #print(sigma)
        #print("--------------------")

        #Utilising the mode selection array to from new array of relevant sigma
        # New sigmas: m1 = row1, m2 = row 2
        new_sigma = np.zeros((length, length))
        #print("new sigma", new_sigma)
        for i in range(0, length):
            new_sigma[i] = sigma.dot(mode_arr[:,i])
            #print(sigma.dot(mode_arr[:,i]))
        #print("new_sigma")
        #print(new_sigma)
        #print("--------------------")

        # Creating the sum of sigmas array
        sum_sigma_arr = np.zeros((length, len(mass)**2))
        for i in range(0, length):
            sumshift = 0
            for j in range(0, length):
                #print("eigvec all =", eigvecs)
                #print("eigvec =", eigvecs[j])
                #print("newsigma ()", new_sigma[i][j])
                #print("-----")
                shift = eigvecs[j] * new_sigma[i][j] * unit_sort
                #print("shift no. {}".format(j), shift)
                #print("--------")
                sumshift += shift
                #print("sum shift no. {}".format(j), sumshift)   
                #print("--------")
            sum_sigma_arr[i][:] = sumshift
            #print("sum_sigma_arr")
            #print(sum_sigma_arr)
            #print("--------------------")
        #print("final sum_sigma_arr")
        #print(sum_sigma_arr)
        #print("final sum_sigma_arr, unit sorted")
        #print(sum_sigma_arr * unit_sort)
        #print("--------------------")

        #Combining the original coord with sigma
        shift_coord = np.zeros((length, len(mass)**2))
        temp = np.zeros((length, len(mass)**2))
        coord = np.array(coord)
        #print("Origin", coord)
        for i in range(0, length):
            tempcoord = coord.reshape(1, len(mass)**2)
            #print("tempcoord")
            #print(tempcoord)
            #print("sum sigma")
            #print(sum_sigma_arr[i])
            shift_coord[i] = tempcoord + (sum_sigma_arr[i])
            #print("new shift")
            #print(shift_coord[i])
            #print("--------------------")
            ##temp = np.around(shift_coord, decimals=7)
        # Allow print to view comparisons (ensure difference)
        #print("Original Coords")
        #print(coord)
        #print("Shift Coords")
        #print(shift_coord)
        #print(temp[0])
        #print(temp[1])
        #print(temp[2])
        #print(coord)
        #print("--------------------")

        # Creating all rotational constants
        # Requires the "rot_inertia_rcom" function to be present
        all_rot_const = np.zeros((length, 3))
        for i in range(0, length):
            temp_coord = np.copy(shift_coord[i])
            final_coord = temp_coord.reshape((len(mass),len(mass)))
            #print("fin Coord",final_coord, "mode", i)
            B, I, COMcoord = rot_inertia_rcom(final_coord, mass, COM)
            #print("COMcoord", COMcoord)
            all_rot_const[i] = B
            #print("B intermed")
            #print(all_rot_const)
            #print("------------------")
        #print(all_rot_const)
    
    return all_rot_const 

.

## [E.14] J Max Function

Take the average $J_{max}$ from the three rotational constants, A, B and C.

\begin{equation}
J_{max, n} = \sqrt{\frac{kT}{2hc\times 0.5(A + C)}} - 0.5
\end{equation}

\begin{equation}
J_{max} = \sum J_{max, n}
\end{equation}

In [4]:
def J_max_func(B, T):
    
    if len(B) != 3:
        print("error")
    else:
        A_rot = B[0]
        B_rot = B[1]
        C_rot = B[2]

    J_max= int(round(np.sqrt((k * T)/(2*(A_rot/2  + C_rot/2)*h*c)) - 0.5))
    
    J_final = np.arange(J_max)
        
    return J_final, A_rot, B_rot, C_rot

.

# PANDORA

## [P.1] Asymmetry Parameter, $\kappa$

Asymmetry Parameter $\kappa$:

\begin{equation}
\label{eq:asymmetry_param}
    \kappa = \frac{2B - A - C}{A - C}
\end{equation}

In [18]:
def asy_k(A, B, C):
    k = ((2 * B) - A - C)
    k = k / (A - C)
    #print("asy_k", k)
    return k

## [P.2] Abbreviation Equation, $f\left(J, n\right)$

The values of $f\left(J, n\right)$ are given by Table 1 in 43KiHaCr.

\begin{equation}
\label{eq:asy_matrix_abbrev}
    f\left(J, n\right) = f\left(J,-n\right) = \frac{1}{4}\left[J\left(J+1\right) - n\left(n+1\right)\right] \times \left[ J\left(J+1\right) - n\left(n-1\right)\right]
\end{equation}

In [19]:
def abbrev(J, n):
    ab = .25*(J*(J+1) - n*(n+1)) * (J*(J+1) - n*(n-1))
    if n == 1:
        ab = ab*2
    return ab

.

## [P.3] Sub Level Calculations, E$_\tau$ 

\begin{equation}
\label{eq:asyk_matrix}
    \textbf{E} \left(\kappa\right) =  
\begin{bmatrix}
E_{-K;-K} & 0 & E_{-K;-K+2} & \cdots & 0
\\[3pt]
0 & E_{-K+1;-K+1} & 0 & \cdots & 0
\\[3pt]
E_{-K;-K+2} & 0 & E_{-K+2;-K+2} & \cdots & 0
\\[3pt]
\vdots & \vdots & \vdots & \ddots & \vdots
\\[3pt]
0 & E_{K;K-2} & 0 & \cdots & E_{K;K}
\end{bmatrix}
\end{equation}


Eigen values obtained by diagonilising the matrix with the following elements:
\begin{equation}
\label{eq:asy_matrix_E(K,K)}
    E_{K;K} = F\left[J\left(J+1\right) - K^2\right] + GK^2
\end{equation}
\begin{equation}
\label{eq:asy_matrix_E(K,K+2)}
    E_{K;K+2} = E_{K+2;K} = H\left[f\left(J,K+1\right)\right]^{\frac{1}{2}}
\end{equation}

The values of $F$, $G$, $H$ in terms of $\kappa$ or $\delta$ depend on the particular identifications of $P_a$, $P_b$, $P_c$ with $P_x$, $P_y$, $P_z$.

In [1]:
def e_matrix(J, asy_k, typing):
    
    g = 2*J + 1
    K = np.arange(-J, J+1)
    #print("Init")
    #print("K", K)
    #print("J", J)
    #print("----------------------------------")
    
    # typing 1,2,3 == 1r, 2r, 3r
    # typing 4,5,6 == 1l, 2l, 3l
    if typing == 1:
        F = .5*(asy_k - 1)
        G = 1
        H = -.5*(asy_k + 1) 
    elif typing == 2:
        F = 0
        G = asy_k
        H = 1
    elif typing == 3:
        F = .5*(asy_k + 1)
        G = 1
        H = .5*(asy_k - 1) 
    elif typing == 4:
        F = .5*(asy_k - 1)
        G = 1
        H = .5*(asy_k + 1)
    elif typing == 5:
        F = 0
        G = asy_k
        H = -1
    elif typing == 6:
        F = .5*(asy_k + 1)
        G = 1
        H = -.5*(asy_k - 1)
    
    e_M = np.zeros((g, g))
    #print("init_M", e_M)
    
    #Diagonal Elements
    #print("Diag Elements")
    for i in np.arange(len(K)):
        #print("i", i, "e", F*(J*(J+1) - K[i]**2) + G*K[i]**2)
        #print("K?", K[i])
        e_M[i][i] = F*(J*(J+1) - K[i]**2) + G*K[i]**2
        #print("---------")
    #print("2nd_M", e_M)
    #print("----------------------------------")
    
    if J == 0:
        pass
        
    else:
        for j in np.arange(g-2):
            #print("n range",np.arange((g-2)) )
            #print("Non diag")
            n = K[j]+1
            if n == 1:
                e_M[j][j+2] = e_M[j+2][j] = 2 * H * abbrev(J, n)**.5
            else:
                #print("n", n, "J", J, "e_M", H * abbrev(J, n)**.5)
                #print("coord", j, j+2)
                #print("j ==", j)
                e_M[j][j+2] = e_M[j+2][j] = H * abbrev(J, n)**.5
                #print("i_M", e_M)
                #print("----------------------------------")
    #print("3rd_M", e_M)
    
    #print("----------------------------------")
    eigvals, eigvecs = la.eig(e_M)
    eigvals = eigvals.real
    eigvals_fin = np.sort(eigvals)
    #print("Total Values", len(eigvals_fin))
    #print("----------------------------------")
    
    return eigvals_fin

.

## [P.4] Sublevel Selection Rules

First requires the creation of the KaKc / KpKo / K-1K1 indexing, then requires the application of specific selection rules depending on the vibration type and parity of the indexing, to then determine which transitions are possible.

### [P.4.1] Sublevel Indexing

In [1]:
def kpko_and_symlevels(j):
    degeneracy = j
    K_temp = []
    for i in range(0, degeneracy, 1):
        K_temp.append([i, i+1])

    flatten_list = np.ravel(K_temp)
    K_final = np.append(flatten_list, degeneracy)

    #prolate, K_p, K_a, K_{-1}, kappa = -1
    K_p = K_final
    #oblate, K_o, K_c, K_{+1}, kappa = +1
    K_o = K_final[::-1]
    
    kpko = np.stack((K_p, K_o), axis=1)
    #print(kpko)
    
    temp=[]
    for i in kpko:
        symlevel_i = np.where((i % 2) == 0, "+", "-")
        temp.append(symlevel_i)
    temp = np.apply_along_axis(lambda d: d[0] + '' + d[1], 1, temp)
    symlevel = np.array(temp)
    #print(symlevel)

    return kpko, symlevel

### [P.4.2] Sublevel Potential Transitions 

In [ ]:
def trans(molecule, lower, lower_index, upper, typing):
    
    # molecule refers to any special spin effects leading to ommissions
    # 0 = normal
    # 1 = odd (nu3 diff = even, e.g. ozone_sym_68+ == 1)
    # 2 = odd (nu3 diff = odd, e.g. ozone_sym_68- == 1)
    
    #Infrared Rules
    final = []
    
    # " Level (Lower), symlevel_lower needs to be locked to one level
    kpko_lower, symlevel_lower_temp = kpko_and_symlevels(lower)
    symlevel_lower = symlevel_lower_temp[lower_index]  
    
    # ' Level (Upper), symlevel_upper can be any for loop scan
    kpko_upper, symlevel_upper = kpko_and_symlevels(upper)
    
    branch = upper - lower
    #print(branch)
    
    boundary = 2 # lower number == higher transitions less possible
    
    #print("-----------")       
    
    if typing == "A":
        
        #Ba: ee-eo, oe-oo
        #Ba_kp = [0]
        #Ba_ko = [-1, +1]
        
        dif_Ba_kp = [0]
        if branch == 1 or branch == 2:
            dif_Ba_ko = [1]
        elif branch == -1 or branch == -2:
            dif_Ba_ko = [-1]
        elif branch == 0:
            dif_Ba_ko = [-1, 1]
        else:
            print("error with branch")
            
        if symlevel_lower == "++":
            if molecule != 1:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "+-"]

                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]
                    #print("J number", lower)
                    #print("lower kpko", kpko_lower[lower_index])
                    #print("upper kpko", kpko_upper[i])
                    #print("deltakp", dif_kp, "deltako", dif_ko)
                    #print("--------")

                    # For Q Branch
                    if branch == 0:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko[1]: 
                            #print("001 - 000  ", upper, kpko_upper[i][0], kpko_upper[i][1],"  ", lower, kpko_lower[lower_index][0], kpko_lower[lower_index][1])
                            if kpko_lower[lower_index][1] <= int((lower)/2 + boundary):
                                final.append(i) 
                            else:
                                pass
                            #print("-------------------")
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            #print("001 - 000  ", upper, kpko_upper[i][0], kpko_upper[i][1],"  ", lower, kpko_lower[lower_index][0], kpko_lower[lower_index][1])
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass
        
        if symlevel_lower == "+-":
            if molecule != 2:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "++"]


                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]

                    # For Q Branch
                    if branch == 0:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko[1]:
                            if kpko_lower[lower_index][1] <= int((lower)/2 + boundary):
                                final.append(i)
                            else:
                                pass
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass
        
        if symlevel_lower == "--":
            if molecule != 1:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "-+"]

                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]
                    #print("J number", lower)
                    #print("lower kpko", kpko_lower[lower_index])
                    #print("upper kpko", kpko_upper[i])
                    #print("deltakp", dif_kp, "deltako", dif_ko)
                    #print("--------")


                    # For Q Branch
                    if branch == 0:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko[1]:
                            #print("001 - 000  ", upper, kpko_upper[i][0], kpko_upper[i][1],"  ", lower, kpko_lower[lower_index][0], kpko_lower[lower_index][1])
                            if kpko_lower[lower_index][1] <= int((lower)/2 + boundary):
                                final.append(i) 
                            else:
                                pass
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            #print("001 - 000  ", upper, kpko_upper[i][0], kpko_upper[i][1],"  ", lower, kpko_lower[lower_index][0], kpko_lower[lower_index][1])
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass
        
        if symlevel_lower == "-+":
            if molecule != 2:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "--"]

                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]

                    # For Q Branch
                    if branch == 0:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko[1]:
                            if kpko_lower[lower_index][1] <= int((lower)/2 + boundary):
                                final.append(i) 
                            else:
                                pass
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass
    
    if typing == "B":
        #Bb: ee-oo, eo-oe
        #Bb_kp = [-1, +1]
        #Bb_ko = [-1, +1]
    
        if branch == 1 or branch == 2:
            dif_Ba_kp = [1]
            dif_Ba_ko = [1]
        elif branch == -1 or branch == -2:
            dif_Ba_kp = [-1]
            dif_Ba_ko = [-1]
        elif branch == 0:
            dif_Ba_kp = [-1, 1]
            dif_Ba_ko = [-1, 1]
        else:
            print("error with branch")    
            
        if symlevel_lower == "++":
            if molecule != 2:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "--"]

                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]
                    #print("lower kpko", kpko_lower[lower_index])
                    #print("upper kpko", kpko_upper[i])
                    #print("deltakp", dif_kp, "deltako", dif_ko)
                    #print("--------")

                    # For Q Branch
                    if branch == 0:
                        if dif_kp == dif_Ba_kp[0] and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp[0] and dif_ko == dif_Ba_ko[1] or dif_kp == dif_Ba_kp[1] and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp[1] and dif_ko == dif_Ba_ko[1]:
                            final.append(i)
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass
        
        if symlevel_lower == "+-":
            if molecule != 1:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "-+"]

                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]
                    #print("lower kpko", kpko_lower[lower_index])
                    #print("upper kpko", kpko_upper[i])
                    #print("deltakp", dif_kp, "deltako", dif_ko)
                    #print("--------")

                    # For Q Branch
                    if branch == 0:
                        if dif_kp == dif_Ba_kp[0] and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp[0] and dif_ko == dif_Ba_ko[1] or dif_kp == dif_Ba_kp[1] and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp[1] and dif_ko == dif_Ba_ko[1]:
                            final.append(i)
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass

        if symlevel_lower == "--":
            if molecule != 2:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "++"]
                
                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]
                    #print("lower kpko", kpko_lower[lower_index])
                    #print("upper kpko", kpko_upper[i])
                    #print("deltakp", dif_kp, "deltako", dif_ko)
                    #print("--------")

                    # For Q Branch
                    if branch == 0:
                        if dif_kp == dif_Ba_kp[0] and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp[0] and dif_ko == dif_Ba_ko[1] or dif_kp == dif_Ba_kp[1] and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp[1] and dif_ko == dif_Ba_ko[1]:
                            final.append(i)
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass
        
        if symlevel_lower == "-+":
            if molecule != 1:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "+-"]
        
                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]
                    #print("lower kpko", kpko_lower[lower_index])
                    #print("upper kpko", kpko_upper[i])
                    #print("deltakp", dif_kp, "deltako", dif_ko)
                    #print("--------")

                    # For Q Branch
                    if branch == 0:
                        if dif_kp == dif_Ba_kp[0] and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp[0] and dif_ko == dif_Ba_ko[1] or dif_kp == dif_Ba_kp[1] and dif_ko == dif_Ba_ko[0] or dif_kp == dif_Ba_kp[1] and dif_ko == dif_Ba_ko[1]:
                            final.append(i)
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass
    
    if typing == "C":
        #Bc: ee-oe, eo-oo
        #Bc Kp = [-1, +1]
        #Bc Ko = [0, 0]
        
        dif_Ba_ko = [0]
        if branch == 1 or branch == 2:
            dif_Ba_kp = [1]
        elif branch == -1 or branch == -2:
            dif_Ba_kp = [-1]
        elif branch == 0:
            dif_Ba_kp = [-1, 1]
        else:
            print("error with branch")
              
        if symlevel_lower == "++":
            if molecule != 1:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "-+"]
                
                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]
                    #print("lower kpko", kpko_lower[lower_index])
                    #print("upper kpko", kpko_upper[i])
                    #print("deltakp", dif_kp, "deltako", dif_ko)
                    #print("--------")

                    # For Q Branch
                    if branch == 0:
                        if dif_ko == dif_Ba_ko and dif_kp == dif_Ba_kp[0] or dif_ko == dif_Ba_ko and dif_kp == dif_Ba_kp[1]:
                            if kpko_lower[lower_index][0] <= int((lower)/2 + boundary):
                                final.append(i) 
                            else:
                                pass
                            #print("-------------------")
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass
        
        if symlevel_lower == "+-": 
            final = []
            if molecule != 2:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "--"]

                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]

                    # For Q Branch
                    if branch == 0:
                        if dif_ko == dif_Ba_ko and dif_kp == dif_Ba_kp[0] or dif_ko == dif_Ba_ko and dif_kp == dif_Ba_kp[1]:    
                            if kpko_lower[lower_index][0] <= int((lower)/2 + boundary):
                                final.append(i) 
                            else:
                                pass
                            #print("-------------------")
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass

        if symlevel_lower == "--": 
            if molecule != 1:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "+-"]
                
                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]

                    # For Q Branch
                    if branch == 0:
                        if dif_ko == dif_Ba_ko and dif_kp == dif_Ba_kp[0] or dif_ko == dif_Ba_ko and dif_kp == dif_Ba_kp[1]:    
                            if kpko_lower[lower_index][0] <= int((lower)/2 + boundary):
                                final.append(i) 
                            else:
                                pass
                            #print("-------------------")
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass
        
        if symlevel_lower == "-+":
            if molecule != 2:
                index_temp = [i for i, e in enumerate(symlevel_upper) if e == "++"]
                
                for i in index_temp: # "i" describes the elements of the available trans
                    dif_kp = kpko_upper[i][0] - kpko_lower[lower_index][0]
                    dif_ko = kpko_upper[i][1] - kpko_lower[lower_index][1]

                    # For Q Branch
                    if branch == 0:
                        if dif_ko == dif_Ba_ko and dif_kp == dif_Ba_kp[0] or dif_ko == dif_Ba_ko and dif_kp == dif_Ba_kp[1]:   
                            if kpko_lower[lower_index][0] <= int((lower)/2 + boundary):
                                final.append(i) 
                            else:
                                pass
                            #print("-------------------")
                        else:
                            pass

                    # For non-Q branches    
                    elif branch == 1 or branch == -1 or branch == 2 or branch == -2:
                        if dif_kp == dif_Ba_kp and dif_ko == dif_Ba_ko:
                            final.append(i)
                        else:
                            pass

                    # Broken branch selection failsafe
                    else:
                        pass
            else:
                pass
            
    return final

## [P.5] Branch Calculations 

Here are the five branches, O, P, Q, R and S.

### [P.5.O] O Branch, $\Delta$ J = -2  

\begin{equation}
\label{eq:O_branch_final}
\bar{\nu_{O}}  = \omega_0 + \frac{1}{2}\left[\left(A_1 + C_1\right)J'\left(J' + 1\right) - \left(A_0 + C_0\right)\left(J' + 2\right)\left(J' + 3\right)\right] \\
+ \frac{1}{2}\left[\left(A_1 - C_1\right)E_{J'} - \left(A_0 - C_0\right)E_{J''}\right]
\end{equation}

In [ ]:
def asytop_v_O(molecule, J, typing, omega, A_0, A_1, B_0, B_1, C_0, C_1):
    
    v_O = []
    v_O_rot = []
    J_index = []
    
    asy_k_II = asy_k(A_0, B_0, C_0)
    asy_k_I = asy_k(A_1, B_1, C_1)
    
    for i in np.arange(len(J)):
        #print("Loop no. {}".format(i))
        
        E_J_II = e_matrix(i+2, asy_k_II, 1)
        E_J_I = e_matrix(i, asy_k_I, 1)
        #print("E_J_II", E_J_II)
        #print("E_J_I", E_J_I)
        
        II_index = np.arange(len(E_J_II))
        
        #print("------------------------")
        
        for j in II_index:
            
            if asy_k_II <= -0.01 or asy_k_II >= 0.01:
                potential_trans = trans(molecule, i+2, j, i, typing)
                potential_trans = np.array(potential_trans)
                #print("potential trans", potential_trans)
            else:
                print("not yet coded")
            
            j = np.array([j]) #Solves single value array issue
            
            for j,k in itertools.product(j,potential_trans):
                
                v_O_temp = omega + .5*((A_1 + C_1)*i*(i + 1) - (A_0 + C_0)*(i + 2)*(i + 3)) + (.5*(A_1 - C_1))*E_J_I[k] - (.5*(A_0 - C_0))*E_J_II[j]
                v_O_rot_temp = (.5*(A_0 + C_0)*(i + 2)*(i + 3) + (.5*(A_0 - C_0))*E_J_II[j]) * h * c_cm
                v_O.append(v_O_temp)
                v_O_rot.append(v_O_rot_temp) 
                J_index.append(i+2)
                    
        #print("--------------------------------------------------")
    #print("--------------------------------------------------")
    return v_O, v_O_rot, J_index

.

### [P.5.P] P Branch, $\Delta$ J = -1  

\begin{equation}
\label{eq:P_branch_final}
\bar{\nu_{P}}  = \omega_0 + \frac{1}{2}\left(J' + 1\right)\left[\left(A_1 + C_1\right)J' - \left(A_0 + C_0\right)\left(J' + 2\right)\right] \\
+ \frac{1}{2}\left[\left(A_1 - C_1\right)E_{J'} - \left(A_0 - C_0\right)E_{J''}\right]
\end{equation}

In [1]:
def asytop_v_P(molecule, J, typing, omega, A_0, A_1, B_0, B_1, C_0, C_1):
    
    v_P = []
    v_P_rot = []
    J_index = []
    
    
    asy_k_II = asy_k(A_0, B_0, C_0)
    asy_k_I = asy_k(A_1, B_1, C_1)
    
    for i in np.arange(len(J)):
        #print("J no. {}".format(i))
        
        E_J_II = e_matrix(i+1, asy_k_II, 1)
        E_J_I = e_matrix(i, asy_k_I, 1)
        #print("E_J_II", E_J_II)
        #print("E_J_I", E_J_I)
        
        II_index = np.arange(len(E_J_II))
        
        #print("------------------------")
        
        for j in II_index:

            if asy_k_II <= -0.01 or asy_k_II >= 0.01:
                potential_trans = trans(molecule, i+1, j, i, typing)
                potential_trans = np.array(potential_trans)
                #print("potential trans", potential_trans)
            else:
                print("not yet coded")
            
            j = np.array([j]) #Solves single value array issue

            for j,k in itertools.product(j,potential_trans):
                v_P_temp = v_P_temp = omega + .5*(i + 1)*((A_1 + C_1)*i - (A_0 + C_0)*(i + 2)) + (.5*(A_1 - C_1))*E_J_I[k] - (.5*(A_0 - C_0))*E_J_II[j]
                v_P_rot_temp = (.5*(A_0 + C_0)*(i + 1)*(i + 2) + (.5*(A_0 - C_0))*E_J_II[j]) * h * c_cm
                v_P.append(v_P_temp)
                v_P_rot.append(v_P_rot_temp)
                J_index.append(i+1)
    
                    
        #print("--------------------------------------------------")
    #print("--------------------------------------------------")
    return v_P, v_P_rot, J_index

.

### [P.5.Q] Q Branch, $\Delta$ J = 0 

\begin{equation}
\label{eq:Q_branch_final}
\bar{\nu_{Q}}  = \omega_0 + \frac{1}{2}J'\left(J' + 1\right)\left[\left(A_1 + C_1\right) - \left(A_0 + C_0\right)\right] \\
+ \frac{1}{2}\left[\left(A_1 - C_1\right)E_{J'} - \left(A_0 - C_0\right)E_{J''}\right]
\end{equation}

In [ ]:
def asytop_v_Q(molecule, J, typing, omega, A_0, A_1, B_0, B_1, C_0, C_1):
    
    v_Q = []
    v_Q_rot = []
    J_index = []
    
    asy_k_II = asy_k(A_0, B_0, C_0)
    #print("asy k", asy_k_II)
    asy_k_I = asy_k(A_1, B_1, C_1)
    
    for i in np.arange(len(J)):
        #print("J no. {}".format(i))
        
        E_J_II = e_matrix(i, asy_k_II, 1)
        E_J_I = e_matrix(i, asy_k_I, 1)
        #print("E_J_II", E_J_II)
        #print("E_J_I", E_J_I)
        
        II_index = np.arange(len(E_J_II))
        
        #print("------------------------")
        
        for j in II_index:
            
            if asy_k_II <= -0.01 or asy_k_II >= 0.01:
                potential_trans = trans(molecule, i, j, i, typing)
                potential_trans = np.array(potential_trans)
                #print("potential trans", potential_trans)
            else:
                print("not yet coded")
            
            j = np.array([j]) #Solves single value array issue
        
            for j,k in itertools.product(j,potential_trans):
                v_Q_temp = omega + .5*i*(i + 1)*((A_1 + C_1) - (A_0 + C_0)) + (.5*(A_1 - C_1))*E_J_I[k] - (.5*(A_0 - C_0))*E_J_II[j]
                v_Q_rot_temp = (.5*(A_0 + C_0)*i*(i + 1) + (.5*(A_0 - C_0))*E_J_II[j]) * h * c_cm
                v_Q.append(v_Q_temp)
                v_Q_rot.append(v_Q_rot_temp) 
                J_index.append(i)
            
            #print("---------------")
                    
            
        #print("--------------------------------------------------")
    #print("--------------------------------------------------")
    return v_Q, v_Q_rot, J_index

.

### [P.5.R]  R Branch, $\Delta$ J = +1

\begin{equation}
\label{eq:R_branch_final}
\bar{\nu_{R}}  = \omega_0 + \frac{1}{2}\left(J'' + 1\right)\left[\left(A_1 + C_1\right)\left(J'' + 2\right) - \left(A_0 + C_0\right)J''\right] \\
+ \frac{1}{2}\left[\left(A_1 - C_1\right)E_{J'} - \left(A_0 - C_0\right)E_{J''}\right]
\end{equation}

In [ ]:
def asytop_v_R(molecule, J, typing, omega, A_0, A_1, B_0, B_1, C_0, C_1):
    
    v_R = []
    v_R_rot = []
    J_index = []
    
    asy_k_II = asy_k(A_0, B_0, C_0)
    asy_k_I = asy_k(A_1, B_1, C_1)
    
    for i in np.arange(len(J)):
        #print("Loop no. {}".format(i))
        
        E_J_II = e_matrix(i, asy_k_II, 1)
        E_J_I = e_matrix(i+1, asy_k_I, 1)
        #print("E_J_II", E_J_II)
        #print("E_J_I", E_J_I)
        
        II_index = np.arange(len(E_J_II))
        
        #print("------------------------")
        
        for j in II_index:
            
            if asy_k_II <= -0.01 or asy_k_II >= 0.01:
                potential_trans = trans(molecule, i, j, i+1, typing)
                potential_trans = np.array(potential_trans)
                #print("potential trans", potential_trans)
            else:
                print("not yet coded")
            
            j = np.array([j]) #Solves single value array issue
            
            for j,k in itertools.product(j,potential_trans):
                v_R_temp = omega + .5*(i + 1)*((A_1 + C_1)*(i + 2) - (A_0 + C_0)*i) + (.5*(A_1 - C_1)*E_J_I[k]) - (.5*(A_0 - C_0)*E_J_II[j])
                v_R_rot_temp = (.5*(A_0 + C_0)*(i + 1)*(i) + (.5*(A_0 - C_0))*(E_J_II[j])) * h * c_cm
                v_R.append(v_R_temp)
                v_R_rot.append(v_R_rot_temp)
                J_index.append(i)
                    
        #print("--------------------------------------------------")
    #print("--------------------------------------------------")
    return v_R, v_R_rot, J_index

.

### [P.5.S] S Branch, $\Delta$ J = +2 

\begin{equation}
\label{eq:S_branch_final}
\bar{\nu_{S}}  = \omega_0 + \frac{1}{2}\left[\left(A_1 + C_1\right)\left(J'' + 2\right)\left(J'' + 3\right) - \left(A_0 + C_0\right)J''\left(J'' + 1\right)\right] \\
+ \frac{1}{2}\left[\left(A_1 - C_1\right)E_{J'} - \left(A_0 - C_0\right)E_{J''}\right]
\end{equation}

In [ ]:
def asytop_v_S(molecule, J, typing, omega, A_0, A_1, B_0, B_1, C_0, C_1):
    
    v_S = []
    v_S_rot = []
    J_index = []
    
    asy_k_II = asy_k(A_0, B_0, C_0)
    asy_k_I = asy_k(A_1, B_1, C_1)
    
    for i in np.arange(len(J)):
        #print("Loop no. {}".format(i))
        
        E_J_II = e_matrix(i, asy_k_II, 1)
        E_J_I = e_matrix(i+2, asy_k_I, 1)
        #print("E_J_II", E_J_II)
        #print("E_J_I", E_J_I)

        II_index = np.arange(len(E_J_II))
        
        #print("------------------------")
        
        for j in II_index:
            
            if asy_k_II <= -0.01 or asy_k_II >= 0.01:
                potential_trans = trans(molecule, i, j, i+2, typing)
                potential_trans = np.array(potential_trans)
                #print("potential trans", potential_trans)
            else:
                print("not yet coded")
            
            j = np.array([j]) #Solves single value array issue
        
            for j,k in itertools.product(j,potential_trans):
                v_S_temp = omega + .5*((A_1 + C_1)*(i + 2)*(i + 3) - (A_0 + C_0)*i*(i + 1)) + (.5*(A_1 - C_1))*E_J_I[k] - (.5*(A_0 - C_0))*E_J_II[j]
                v_S_rot_temp = (.5*(A_0 + C_0)*(i + 1)*i + (.5*(A_0 - C_0))*(E_J_II[j])) * h * c_cm
                v_S.append(v_S_temp)
                v_S_rot.append(v_S_rot_temp)
                J_index.append(i)
                
                    
        #print("--------------------------------------------------")
    #print("--------------------------------------------------")
    return v_S, v_S_rot, J_index

.

## [P.6] Normalising the Intensities (Mode Constrained)

Restricted between modes at this point, purely evalutation the potential intensities for PQR. 

The Boltzmann Distribution is written as:
\begin{equation}\label{eq:21}
    \frac{N_J}{N} = \frac{g_J e^{\left(\frac{-E_J}{kT}\right)}}{f}
\end{equation}
Where, the degeneracy, $g_J$, the energy of a given level $E_J$, and the partition function, $f$ are described as:
\begin{equation}\label{eq:22}
    g_J = (2J+1)
\end{equation}

\begin{equation}\label{eq:23}
    E_J = F_J
\end{equation}

\begin{equation}\label{eq:24}
    f = \sum g_J e^{\left(\frac{-E_J}{kT}\right)}
\end{equation}

The full equation used was expressed as the following:

\begin{equation}
\label{eq:boltz_distribution_expanded}
    \frac{N_J}{N} =\frac{(2J+1) e^{\left(\frac{\frac{1}{2}\left(A + C\right)J\left(J + 1\right) + \frac{1}{2}\left(A - C\right)E_\tau}{kT}\right)}}{f}
\end{equation}


In [1]:
def Norm_I(P_rot, P_index, Q_rot, Q_index, R_rot, R_index, T, scenario, weighting):
    
    new_F_P = []
    new_F_Q = []
    new_F_R = []
            
    for i in np.arange(len(P_rot)):
        #Defining E_j, the rotational energy [J]
        #Ground State (Ej'')
        E_J = P_rot[i]
        #print("Energy",E_J)

        #Degeneracy
        g_j = (2*P_index[i]) + 1

        #Boltzmann Distribution of the rotational energy levels
        #Ground State (BD'')
        B_D = g_j * np.exp(np.float((-E_J) / (k * T)))
        #print(B_D)
        
        new_F_P.append(B_D)
    
    for i in np.arange(len(Q_rot)):
        #Defining E_j, the rotational energy [J]
        #Ground State (Ej'')
        E_J = Q_rot[i]
        #print("Energy",E_J)

        #Degeneracy
        g_j = (2*Q_index[i]) + 1

        #Boltzmann Distribution of the rotational energy levels
        #Ground State (BD'')
        B_D = g_j * np.exp(np.float((-E_J) / (k * T))) 
        #print(B_D)
        
        new_F_Q.append(B_D)
        
    for i in np.arange(len(R_rot)):
        #Defining E_j, the rotational energy [J]
        #Ground State (Ej'')
        E_J = R_rot[i] 
        #print("Energy",E_J)

        #Degeneracy
        g_j = (2*R_index[i]) + 1

        #Boltzmann Distribution of the rotational energy levels
        #Ground State (BD'')
        B_D = g_j * np.exp(np.float((-E_J) / (k * T)))
        #print(B_D)
        
        new_F_R.append(B_D)
        
    
    #The sum of all the distributions
    #Ground State (Z(T)''
    if scenario == 1:
        
        Z = sum(new_F_P) + sum(new_F_Q) + sum(new_F_R)
        f_j_P = (new_F_P / Z)
        f_j_Q = (new_F_Q / Z)
        f_j_R = (new_F_R / Z)
        f_j_PQR = np.array([f_j_P,f_j_Q, f_j_R])
        f_j_PQR = f_j_PQR * weighting
        
        return f_j_PQR
    
    elif scenario == 2:
        
        Z = sum(new_F_P) + sum(new_F_R)
        f_j_P = (new_F_P / Z)
        f_j_R = (new_F_R / Z)
        f_j_PR = np.array([f_j_P, f_j_R])
        f_j_PR = f_j_PR * weighting
        
        return f_j_PR

.

## [P.7] Final Intensities 

Will work regardless the number of modes present (I hope).

In [28]:
def Fin_I(*arg):
    blank = []
    
    for i in arg:
        blank.append(i)
    
    #print("blank", blank)

    blanknew = np.concatenate(blank, axis=None)
    result = blanknew.flatten()
    #print(result)
    final = sum(result)
    #print(final)
    
    final_I = np.array(blank)
    #print("og", final_I)
    #print("--------")
    for i in np.arange(len(final_I)):
        final_I[i] = final_I[i]/final
        #print(random)
        #print("--------")
    #print("final", final_I)
    
    return final_I

.

## [A] Appendix Functions

### [A.1] Significant Figures Definer 

In [29]:
#Taken from: https://stackoverflow.com/questions/58531091/how-to-do-element-wise-rounding-of-numpy-array-to-first-non-zero-digit
def significant_figures(arr, num=1):
    # : compute the order of magnitude
    order = np.zeros_like(arr)  
    mask = arr != 0
    order[mask] = np.floor(np.log10(np.abs(arr[mask])))
    del mask  # free unused memory
    # : compute the corresponding precision
    prec = num - order - 1
    return np.round(arr * 10.0 ** prec) / 10.0 ** prec

### [A.2] Hertz to Wavenumbers 

In [30]:
def Hz_to_wn(hertz):
    wn = hertz / c * 10
    print(wn)
    return wn